In [1]:
#Installs
!pip install tensorflow scikit-learn pandas pandasql sqlalchemy

In [3]:
#Imports
import pandas as pd
import os
from pandasql import sqldf
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np


In [8]:
df= pd.read_csv('diabetes.csv')

In [1]:
rds_host = 'database-1.czawiquo8nur.us-east-2.rds.amazonaws.com'
username = "admin"
password = "Os.s3$$-ctn69bRxefQtn_Z$V7eM"
db_name = 'cs6440_team82'

# Create the SQLAlchemy engine

db_connection_str = f"mysql+pymysql://{username}:{password}@{rds_host}/{db_name}"
db_connection = create_engine(db_connection_str)

df = pd.read_sql('SELECT * FROM diabetes_data', con=db_connection)

NameError: name 'create_engine' is not defined

In [9]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [11]:
# df = df.drop(['Pregnancies', 'Outcome', 'patientId'], axis=1)
df = df.drop(['Pregnancies', 'Outcome'], axis=1)


In [12]:
df

,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,148,72,35,0,33.6,0.627,50
1,85,66,29,0,26.6,0.351,31
2,183,64,0,0,23.3,0.672,32
3,89,66,23,94,28.1,0.167,21
4,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...
763,101,76,48,180,32.9,0.171,63
764,122,70,27,0,36.8,0.340,27
765,121,72,23,112,26.2,0.245,30
766,126,60,0,0,30.1,0.349,47


In [13]:

from sklearn.preprocessing import StandardScaler

X = df.drop(['BMI', 'DiabetesPedigreeFunction'], axis=1)
y = df[['BMI', 'DiabetesPedigreeFunction']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# y_train_scaled = scaler.transform(y_train)
# y_test_scaled = scaler.transform(y_test)



In [256]:

os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_SESSION_TOKEN'] = ''

In [14]:
model_path = 'container/local_custom_model.keras'


In [57]:

# Define a custom TensorFlow model
# def create_model():
#     model = tf.keras.Sequential([
#         tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],), 
#                               kernel_regularizer=tf.keras.regularizers.l2(0.01)),  # Regularization
#         tf.keras.layers.Dropout(0.2),  # Dropout for regularization
#         tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
#         tf.keras.layers.Dense(32, activation='relu'),
#         tf.keras.layers.Dense(2)  # Two outputs for multi-target regression
#     ])
#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mse')
#     return model

def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],),
                              kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(2)  # Two outputs for multi-target regression
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='mse')
    # model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005), loss='mse')

    return model




# Train the model
model = create_model()
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Save the model locally
model.save(model_path)
print("Model saved locally in 'local_custom_model' directory")


Epoch 1/10


/opt/anaconda3/envs/KBAI/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 879us/step - loss: 283.9840
Epoch 2/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 939us/step - loss: 131.1829
Epoch 3/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - loss: 88.2210
Epoch 4/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 952us/step - loss: 72.3426
Epoch 5/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 891us/step - loss: 73.4640
Epoch 6/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 839us/step - loss: 60.2482
Epoch 7/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 65.5115
Epoch 8/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 877us/step - loss: 64.0967
Epoch 9/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 890us/step - loss: 51.1469
Epoch 10/10
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 865us/step - loss: 56.7701
Model saved locally in 'local_custom_model' directory


In [58]:

# Load the model

model = tf.keras.models.load_model(model_path)
# 89	66	23	94	28.1	0.167	21
input_data = np.array([[89,66,23,94,21]]) #Glucose	BloodPressure	SkinThickness	Insulin Age
# Make predictions
predictions = model.predict(input_data)

# Display the results
print("Custom Predictions (BMI, DiabetesPedigreeFunction):", predictions)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Custom Predictions (BMI, DiabetesPedigreeFunction): [[25.870193  -1.1973704]]


In [41]:
df

,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,148,72,35,0,33.6,0.627,50
1,85,66,29,0,26.6,0.351,31
2,183,64,0,0,23.3,0.672,32
3,89,66,23,94,28.1,0.167,21
4,137,40,35,168,43.1,2.288,33
...,...,...,...,...,...,...,...
763,101,76,48,180,32.9,0.171,63
764,122,70,27,0,36.8,0.340,27
765,121,72,23,112,26.2,0.245,30
766,126,60,0,0,30.1,0.349,47


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Custom Predictions (BMI, DiabetesPedigreeFunction): [[32.130245   -0.40065578]]


In [19]:
predictions = model.predict(X_test_scaled)
# Extract predictions
predicted_bmi = predictions[:, 0]  # First column is BMI
predicted_bloodpressure = predictions[:, 1]  # Second column is DiabetesPedigreeFunction

# Example output
print("Predicted BMI:", predicted_bmi[:5])
print("Predicted DiabetesPedigreeFunction:", predicted_bloodpressure[:5])

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Predicted BMI: [0.8129329  0.7272649  0.47917548 0.48349205 0.6336468 ]
Predicted DiabetesPedigreeFunction: [ 0.03245353  0.05206988 -0.06130499  0.01502582  0.0259823 ]


In [20]:
predictions = model.predict(X_train)
# Extract predictions
predicted_bmi = predictions[:, 0]  # First column is BMI
predicted_bloodpressure = predictions[:, 1]  # Second column is DiabetesPedigreeFunction

# Example output
print("Predicted BMI:", predicted_bmi[:5])
print("Predicted DiabetesPedigreeFunction:", predicted_bloodpressure[:5])

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step
Predicted BMI: [17.423794 29.857422 22.26096  27.680525 28.896843]
Predicted DiabetesPedigreeFunction: [-0.32050794 -1.15456    -0.55530566 -0.69421744 -1.650704  ]


In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
# Evaluate on training set
y_train_pred = model.predict(X_train)
train_error = mean_squared_error(y_train, y_train_pred)

# Evaluate on test set
y_test_pred = model.predict(X_test)
test_error = mean_squared_error(y_test, y_test_pred)

print("Training Error:", train_error)
print("Testing Error:", test_error)

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 407us/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 578us/step
Training Error: 31.956268105664005
Testing Error: 36.59844784157924
